## face_recognition 

In [10]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import tensorflow as tf

In [11]:
IMAGE_SIZE = [100, 100]
train_path = 'images'
valid_path = 'valid_image'

In [12]:
#preprocessing layer to the front of vgg16
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [13]:
#dont train existing weight
for layer in vgg.layers:
    layer.trainable =False

In [14]:
#getting number of classes
folder = glob('images/*')

In [15]:
x = Flatten()(vgg.output)
prediction = Dense(len(folder), activation=tf.nn.softmax)(x)

model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100, 100, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 100, 100, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 50, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 50, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 50, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 25, 128)       0         
__________

In [16]:
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=['accuracy']
)

In [17]:
#import image from datasets using ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  featurewise_center = False, #set input mean 0 over the data
                                  samplewise_center = False, #set each sample mean 0
                                  featurewise_std_normalization=False, #divide inputs by std of the datasets
                                  zca_whitening = False,
                                  rotation_range=10, #randomly rotate images in the range 0-100
                                  width_shift_range = 0.2, #randomly shift image vertically. fraction of total height.
                                  vertical_flip = False
                                  )

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory('images',
                                                target_size=(100,100),
                                                batch_size=32,
                                                class_mode='categorical')

test_set = test_datagen.flow_from_directory('valid_image',
                                            target_size=(100,100),
                                            batch_size=32,
                                            class_mode='categorical')

Found 200 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [18]:
#train model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=32,
  validation_steps=len(test_set)
)

Epoch 1/5
32/32 [==============================] - 349s 11s/step - loss: 0.1951 - acc: 0.9231 - val_loss: 0.0341 - val_acc: 1.0000
Epoch 2/5
32/32 [==============================] - 339s 11s/step - loss: 0.0137 - acc: 1.0000 - val_loss: 0.0192 - val_acc: 1.0000
Epoch 3/5
32/32 [==============================] - 314s 10s/step - loss: 0.0080 - acc: 1.0000 - val_loss: 0.0157 - val_acc: 1.0000
Epoch 4/5
32/32 [==============================] - 303s 9s/step - loss: 0.0057 - acc: 1.0000 - val_loss: 0.0136 - val_acc: 1.0000
Epoch 5/5
32/32 [==============================] - 303s 9s/step - loss: 0.0044 - acc: 1.0000 - val_loss: 0.0118 - val_acc: 1.0000


In [19]:
model.save('model_vgg16.h5')